<a href="https://colab.research.google.com/github/mattocanas/MLM-Guide/blob/main/MaskedLanguageModelingGuide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

To do this example of MLM, we will use the r/askscience subset of ELI5

In [ ]:
from datasets import load_dataset

eli5 = load_dataset("eli5", split="train_asks[:5000]")

Downloading:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/286M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/36.2M [00:00<?, ?B/s]

Dataset eli5 downloaded and prepared to /root/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa. Subsequent calls will reuse this data.


In [ ]:
eli5[0]

{'q_id': 'k4w1z',
 'title': 'What are the long-term effects of using raid?',
 'selftext': "I'm talking about the one you plug into the wall. When it kills insects, it surely can't be good for my lungs and brain, or are the doses simply low enough not to be harmful? What are the long-term effects, then?",
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['c2hk30c'],
  'text': ["I did some research as a layman. One forum user [claims it contains](_URL_1_) 300mg of Transfluthrin, but I can't seem to find any SC Johnson or Raid site that corroborates that. Transfluthrin is a pyrethroid, related to a pesticide produced by Chrysanthemums called pyrethrum. The EPA has [scant information on pyrethroids](_URL_0_), but the consensus seems to be that low doses are mostly harmless."],
  'score': [3]},
 'title_urls': {'url': []},
 'selftext_urls': {'url': []},
 'answers_urls': {'url': ['http://www.epa.gov/pesticides/health/mosquitoes/pyrethroids4mosquitoes.htm',
   'http://www.shan

Split into 80% training, 20% test

In [ ]:
eli5 = eli5.train_test_split(test_size=0.2)

In [ ]:
eli5['train'][0]

{'q_id': '1dirfs',
 'title': 'Whenever I see images from scanning tunneling electron microscopes, there is a grey, fluidy background. What is this background and why is it there?',
 'selftext': "Noticed this as well on the new short animation by IBM using hydrogen atoms.  When the atoms moved, you would see a ripple, like water, around the atom.  It can't be electrons because hydrogen would not have that many rings.  A little help?!",
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['c9qpi35', 'c9qvzon'],
  'text': ["They use all [sorts of colors](_URL_2_) (that image is from IBM as well). It's [carbon on a copper surface](_URL_1_), and yes, the ripples are the [density of surface electrons](_URL_0_). \n\nThis has nothing to do with the concentric electron orbits in the Bohr model of the atom. The Bohr model is wrong; atoms don't look like that.",
   "The animation isn't using H atoms but CO molecules.  \n\nThe ripples are the standing waves of electron density from t

In [ ]:
eli5['test'][0]

{'q_id': '3zu51q',
 'title': "Why does my absinthe suddenly 'louche' at a certain concentration? What makes that concentration critical to the effect?",
 'selftext': '',
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['cyp5nno', 'cypj9fh'],
  'text': ["Ethanol is a better solvent than water, so when you dilute with water, stuff with poor water solubility form solids and turn the drink cloudy. In absinthe it's the oils from the various spices. In whiskey (unless chill-filtered) it's from the wood and malt.\n\nAt what exact concentrations you start seeing things, that's up to the spirit in question. I would dispute that it happens suddenly. Try adding water drop by drop with proper stirring.",
   'Some of the compounds dissolved in the absinthe (solute) are soluble in ethanol (solvent), but poorly so in water. As more water is added, the compounds precipitate, clouding the solution. \n\nThe critical concentration required for this to take place depends on the relative 

We now want to tokenize the dataset. We will do this using the `DistilRoBERTa` tokenizer. Note that the answers text is nested inside of the answers dictionary. To make this easier to get to, we will call the `flatten` method which will extract al the nested subfields:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')

In [ ]:
eli5 = eli5.flatten()
eli5['train'][0]

{'q_id': '1dirfs',
 'title': 'Whenever I see images from scanning tunneling electron microscopes, there is a grey, fluidy background. What is this background and why is it there?',
 'selftext': "Noticed this as well on the new short animation by IBM using hydrogen atoms.  When the atoms moved, you would see a ripple, like water, around the atom.  It can't be electrons because hydrogen would not have that many rings.  A little help?!",
 'document': '',
 'subreddit': 'askscience',
 'answers.a_id': ['c9qpi35', 'c9qvzon'],
 'answers.text': ["They use all [sorts of colors](_URL_2_) (that image is from IBM as well). It's [carbon on a copper surface](_URL_1_), and yes, the ripples are the [density of surface electrons](_URL_0_). \n\nThis has nothing to do with the concentric electron orbits in the Bohr model of the atom. The Bohr model is wrong; atoms don't look like that.",
  "The animation isn't using H atoms but CO molecules.  \n\nThe ripples are the standing waves of electron density from

Now, instead of tokenizing each answer separately, we will convert that list to one big string and tokenize them all together:

In [ ]:
def preprocess_function(examples):
  return tokenizer([" ".join(x) for x in examples['answers.text']])

In [ ]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (584 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (557 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1172 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (599 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (524 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (741 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1292 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (551 > 512). Running this sequence through the model will result in indexing errors


The dataset is no fully tokenized, but there is an issue. Some of the sequences are longer than the max length of the model.

To fix this, we will concatenate all the sequences, and then break them up into shorter chunks, or blocks:

In [ ]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Now we will dynamically pad the dataset and apply masking. This is all done using the datacollator. Note that the dataset should be tokenized before this step. But the collator performs additional online masking

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors='tf')

Now we can fine tune our model on this dataset using MLM!

In [ ]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
from transformers import TFAutoModelForMaskedLM

model = TFAutoModelForMaskedLM.from_pretrained('distilroberta-base')

All PyTorch model weights were used when initializing TFRobertaForMaskedLM.

All the weights of TFRobertaForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


In [ ]:
tf_train_set = model.prepare_tf_dataset(
    lm_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    lm_dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument! Models from Transformers have default task-relevant loss function already

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

Epoch 1/3
573/573 [==============================] - 87s 136ms/step - loss: 2.0353 - val_loss: 1.8272
Epoch 2/3
573/573 [==============================] - 76s 132ms/step - loss: 1.9357 - val_loss: 1.7560
Epoch 3/3
573/573 [==============================] - 76s 132ms/step - loss: 1.8873 - val_loss: 1.7528


In [ ]:
# Define the sequence with a masked token
sequence = "Tiger sharks like to eat <mask>."

# Tokenize the sequence
inputs = tokenizer(sequence, return_tensors="tf")

# Get the model's predictions
outputs = model(inputs)

# The predictions are logits for each token in the vocabulary, for each position in the sequence
predictions = outputs.logits

# The index of the masked token can be found with the tokenizer
mask_index = inputs["input_ids"].numpy()[0].tolist().index(tokenizer.mask_token_id)

# Get the predicted token
predicted_index = tf.argmax(predictions[0, mask_index]).numpy()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

if predicted_token.startswith('Ġ'):
    predicted_token = predicted_token[1:]

print(predicted_token)



meat
